In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
# Loading the dataset
def loaddata(filename):
    df = pd.read_csv(f'/content/drive/MyDrive/{filename}.csv', sep=',', encoding='latin-1')
    return df
def loaddata2():
    df = pd.read_csv(f'/content/drive/MyDrive/data-brm/ratings.csv', sep=',', encoding='latin-1')
    return df

keywords   = loaddata("keywords")
books   = loaddata("cleaned_book_data")
ratings = loaddata2()

In [ ]:
books_for_content_based = books[:]
books_for_collaborative_based = books[:]

Content based

In [ ]:
# Vectorize the keywords summary using TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer = 'word',
                        min_df=3,
                        max_df = 0.6,
                        stop_words="english",
                        encoding = 'utf-8',
                        token_pattern=r"(?u)\S\S+")
tfidf_encoding = tfidf.fit_transform(keywords["keywords"])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
book_cosine_sim = cosine_similarity(tfidf_encoding, tfidf_encoding)

In [ ]:
original_book = books_for_content_based[:]

In [ ]:
books_for_content_based = pd.Series(keywords['title'])

import pandas as pd

def recommend_books_similar_to(book_name, n, cosine_sim_mat=book_cosine_sim):
    # Get index of the input book
    input_idx = books_for_content_based[books_for_content_based == book_name].index

    # Check if the result is empty
    if input_idx.empty:
        return None

    input_idx = input_idx[0]  # Extract the first index

    # Find top n similar books with decreasing order of similarity score
    top_n_books_idx = list(pd.Series(cosine_sim_mat[input_idx]).sort_values(ascending=False).iloc[1:n + 1].index)

    # Get the similarity scores for the recommended books
    similarity_scores = [cosine_sim_mat[input_idx][i] for i in top_n_books_idx]

    books_list = list(books_for_content_based)
    recommended_books = [books_for_content_based[i] for i in top_n_books_idx]

    return recommended_books


Collaborative based

In [ ]:
rating_users = ratings['user_id'].value_counts().reset_index().\
               rename({'index':'user_id','user_id':'rating'}, axis=1)

rating_books = ratings['book_id'].value_counts().reset_index().\
               rename({'index':'book_id','book_id':'rating'}, axis=1)


ratings = ratings[ratings['user_id'].isin(rating_users[rating_users['rating']>5]['user_id'])]
ratings = ratings[ratings['book_id'].isin(rating_books[rating_books['rating']> 2000]['book_id'])]

temp = ratings.head(650000)
ratings = temp

In [ ]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 66.6 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 41.8 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.8 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-rpzqr7us
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-rpzqr7us
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 60a2948ba1745e35ee5209d5e3d0b861aca54495
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.4.0-py3-none-any.whl size=99

In [ ]:
from sklearn import model_selection, metrics, preprocessing
# import required modules
import random
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics, preprocessing
import copy

import torch
from torch import nn, optim, Tensor

from torch_sparse import SparseTensor, matmul

from collections import defaultdict

from torch_geometric.utils import structured_negative_sampling
from torch_geometric.data import download_url, extract_zip
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.typing import Adj
import torch.nn.functional as F

In [ ]:
from google.colab import files

# Use files.upload() to upload a file from your local system
uploaded = files.upload()

# map_location=torch.device('cpu')
# Load the uploaded model file
model_path = 'trained_graph_rec_model.pth'  # Specify the filename
model = torch.load(model_path)


Saving trained_graph_rec_model.pth to trained_graph_rec_model.pth


In [ ]:
ratings.shape

(400000, 3)

In [ ]:
import joblib
# Upload the file from your local machine to Colab
uploaded = files.upload()

# Check if the file was successfully uploaded
if 'label_encoder_books.pkl' in uploaded:
    print("File uploaded successfully!")

# Load the label encoder from the uploaded file
loaded_label_encoder = joblib.load('label_encoder_books.pkl')
lbl_books = loaded_label_encoder

# # Now you can use loaded_label_encoder to transform data as needed
# # For example, if you want to transform a list of labels:
# encoded_labels = loaded_label_encoder.transform(['label1', 'label2', 'label3'])

# # If you want to inverse transform encoded labels to get the original labels:
# original_labels = loaded_label_encoder.inverse_transform(encoded_labels)


Saving label_encoder_books.pkl to label_encoder_books.pkl
File uploaded successfully!


In [ ]:
def recommend_books_for_collaborative(user_id, model, lbl_books, num_recommendations, device):
    # Assuming 'user_id' is the user for whom you want personalized recommendations
    try:
        # Assuming 'user_id' is the user for whom you want personalized recommendations
        user_embedding = model['users_emb.weight'][user_id].to(device)
    except Exception as e:
        # Handle the exception (e.g., print an error message)
        print("Error while accessing user embedding:", e)
        return None, []

    # Get all item embeddings
    all_item_embeddings = model['items_emb.weight'].to(device)

    # print("all_item_embeddings:  ",all_item_embeddings)

    # Compute scores for all items using dot product
    scores = torch.mm(all_item_embeddings, user_embedding.view(-1, 1)).squeeze().cpu().detach().numpy()

    # print("scores: ",scores)

    # Sort the items by their scores in descending order
    sorted_indices = np.argsort(scores, axis=0)[::-1]

    # print("sorted_indices: ",sorted_indices)

    # # Get the top 'num_recommendations' book indices
    top_indices = sorted_indices[:num_recommendations]

    # # Map the book indices back to their original labels
    recommended_books = lbl_books.classes_[top_indices]

    # Create a list of tuples containing book and rating
    recommendations_with_ratings = [(book, score) for book, score in zip(recommended_books, scores[top_indices])]
    print("ratings: ",recommendations_with_ratings)

    return recommended_books,recommendations_with_ratings

Hybrid Book Recommendation System

In [ ]:
import pandas as pd
import random

#loop through each genre and collect all books in a genre
#put all books in a list and remove duplication
#sort in descending order of average rating
#take the first 5 books

#Take each book title and send it through content based(this way i get 25 books in total)
#again sort the books in descending order of average rating
#return the top 5 books

In [ ]:
# Assuming your line of code is:


#NOT TO BE EXECUTED - FOR TRIAL PURPOSE ONLY


genre_series = original_book.head(1)['genre']

# Extract the value from the Series (assuming it's a single string)
genre_string = genre_series.values[0]

# Split the genre string by commas
genres = genre_string.split(', ')

# genres will now be a list containing individual genres
print(genres)


In [ ]:
def filter_books_by_genres(genre_list, book_dataset):
    matching_books = []

    for index, book in book_dataset.iterrows():  # Assuming book_dataset is a DataFrame
        genre_series = book['genre']

        # Extract the value from the Series (assuming it's a single string)
        genre_string = genre_series

        # Split the genre string by commas
        genres = genre_string.split(', ')

        # Check if any of the book's genres are in the genre_list
        if any(genre in genre_list for genre in genres):
            book_id = book['book_id']
            title = book['title']
            rating = float(book['rating'])  # Convert rating to float for proper sorting
            matching_books.append((book_id, title, rating))

    # Sort the list in decreasing order of rating
    matching_books.sort(key=lambda x: x[2], reverse=True)

    return matching_books

In [ ]:
def looped_recommender(title,num):
    # Content-Based Recommendations
    content_based_recommendations = recommend_books_similar_to(title, n=num)

    return content_based_recommendations

In [ ]:
def find_rows_by_titles(book_dataset, book_titles_to_find):
    matching_rows = []

    for index, row in book_dataset.iterrows():
        title = row['title']

        if title in book_titles_to_find:
            matching_rows.append(row)

    if len(matching_rows) > 0:
        # Create a new DataFrame from the matching rows
        matching_df = pd.DataFrame(matching_rows)

        # Reset the index of the new DataFrame
        matching_df.reset_index(drop=True, inplace=True)

        return matching_df
    else:
        return None

In [ ]:
def content_based_for_hybrid_system(list_of_genres,num):

  list_of_books = filter_books_by_genres(list_of_genres,original_book)

  top_books = list_of_books[:num] if len(list_of_books) >= num else list_of_books[:]

  # Initialize a variable to store all recommended books
  all_recommended_books = []

  # Pass each title from top_books to looped_recommender
  for book_info in top_books:
      _, title, _ = book_info
      recommended_books_for_title = looped_recommender(title,num)

      if recommended_books_for_title is None:
        continue

      all_recommended_books.extend(recommended_books_for_title)

  # Ensure there are no duplicates across all recommendations
  all_recommended_books = list(set(all_recommended_books))

  matching_rows = find_rows_by_titles(original_book, all_recommended_books)

  # Sort the DataFrame in descending order of rating
  matching_df_sorted = matching_rows.sort_values(by='rating', ascending=False)

# 'matching_df_sorted' will contain the rows sorted by rating in descending order


  return matching_df_sorted[:num]

In [ ]:
def collaborative_based_for_hybrid_system(user_id, model, lbl_books, num_of_recommendation,device):

  collaborative_based_recommendations, predicted_ratings = recommend_books_for_collaborative(user_id, model, lbl_books, num_of_recommendation,device)
  if collaborative_based_recommendations is None:
        return None
  # print(f"Top {num_of_recommendation} recommended books with ratings for user {user_id}:")
  book_data = []
  for i, (book_id, rating) in enumerate(zip(collaborative_based_recommendations, predicted_ratings)):
      # Find the row in the books_for_collaborative_based DataFrame with the matching book_id
        book_row = books_for_collaborative_based[books_for_collaborative_based['book_id'] == book_id]

        # Check if a matching row was found
        if not book_row.empty:
            # Append the entire row to the book_data list
            book_data.append(book_row.iloc[0])  # Assuming there's only one matching row

    # Create a DataFrame from the list of rows
  recommended_books_df = pd.DataFrame(book_data)
  return recommended_books_df

In [ ]:
user_id = 87
num_of_recommendation = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Content-Based Recommendations
list_of_genres = ['Cultural','Environmental','Business','Animals']
content_based_recommendations = content_based_for_hybrid_system(list_of_genres,num_of_recommendation)

# content_based_recommendations

# Collaborative-Based Recommendations
collaborative_based_recommendations = collaborative_based_for_hybrid_system(user_id, model, lbl_books, num_of_recommendation,device)

# Combine content-based and collaborative-based recommendations
if collaborative_based_recommendations is not None:
  combined_recommendations = pd.concat([content_based_recommendations, collaborative_based_recommendations])
else:
  combined_recommendations = content_based_recommendations

# Print the combined recommendations DataFrame
# Assuming combined_recommendations is a Pandas DataFrame with the specified columns

# Concatenate df1 and df2 vertically
combined_df = pd.concat([content_based_recommendations, collaborative_based_recommendations])

# Print the combined DataFrame
# content_based_recommendations
# collaborative_based_recommendations
combined_df

ratings:  [(417, 5.5364485e-08), (697, 5.1151563e-08), (919, 4.624921e-08)]


,book_id,title,isbn,author,genre,description,image,pages,rating,total_ratings,language
6,9747,A Predictable Tragedy: Robert Mugabe and the C...,081224267X,Daniel Compagnon,"Africa, Cultural",When the southern African country of Rhodesia ...,https://i.gr-assets.com/images/S/compressed.ph...,333,4.22,9,en
4,8001,Soweto Inside Out: Stories About Africa's Famo...,143024590,"Adam Roberts,Joe Thloloe,Tim Butcher","Africa, Cultural, Nonfiction",This is a title about Soweto from inside and o...,https://i.gr-assets.com/images/S/compressed.ph...,237,4.00,21,en
5,9059,Freak Out! My Life with Frank Zappa,859654796,Pauline Butcher,"Autobiography, Biography, Memoir, Music","In 1967, 21-year-old Pauline Butcher was worki...",https://i.gr-assets.com/images/S/compressed.ph...,320,3.94,199,en
416,417,"In the Line of Fire (Uniformly Hot!, #15)",373796021,Jennifer LaBrecque,"Category Romance, Contemporary, Contemporary R...","Subject: Colton Sawyer, Army Officer.Current S...",https://i.gr-assets.com/images/S/compressed.ph...,224,3.81,139,en
696,697,"Green Babies, Sage Moms: The Ultimate Guide to...",045122289X,"Lynda Fassa,Harvey Karp","Environment, Green, Nonfiction, Parenting",From the trailblazing founder of Green Babies ...,https://i.gr-assets.com/images/S/compressed.ph...,234,3.31,143,en
918,919,Krav Maga: How to Defend Yourself Against Arme...,1583940081,Imi Sde-Or,"Combat, Health, Martial Arts, Nonfiction, Sports",Krav Maga is today's cutting edge self-defense...,https://i.gr-assets.com/images/S/compressed.ph...,256,3.93,135,en
